In [ ]:
#default_exp loss

## TODO: Clear losses

In [ ]:
#export
from fastai2.basics import *
from faststyle import *

In [ ]:
#export
def gram(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
#export
def content_loss(pred, targ, fts, layer_ws=None):
  layer_ws = layer_ws or [1.]*len(fts['pred']['cnt'])
  return sum([w*F.mse_loss(p,t) for w,p,t in zip_safe(layer_ws,fts['pred']['cnt'],fts['targ']['cnt'])])

In [ ]:
#export
def style_loss(pred, targ, fts, layer_ws=None, stl_ws=None):
  bs = fts['pred']['stl'][0].shape[0]
  sims_gs = L(torch.stack([gram(f) for f in fs]).wmean(stl_ws, dim=0) for fs in fts['source']['stl'])
  pred_gs = [gram(f) for f in fts['pred']['stl']]
  layer_ws = layer_ws or [1.]*len(pred_gs)
  stl_losses = [w*F.mse_loss(g1.repeat(bs,1,1),g2) for w,g1,g2 in zip_safe(layer_ws,sims_gs,pred_gs)]
  return sum(stl_losses)

In [ ]:
#export
def tv_loss(pred,*_):      
  tv_h = ((pred[:,:,1:,:] - pred[:,:,:-1,:]).pow(2)).mean()
  tv_w = ((pred[:,:,:,1:] - pred[:,:,:,:-1]).pow(2)).mean()    
  return (tv_h + tv_w)

In [ ]:
#export
def pixel_loss(pred, targ, fts, base_loss=F.mse_loss, **kwargs):
  return base_loss(pred,targ,**kwargs)

In [ ]:
#export
@funcs_kwargs
class FastStyleLoss:
  _methods=['stl_loss','cnt_loss','tv_loss']
  def __init__(self, stl_w=3e5, cnt_w=1, tv_w=20, **kwargs):
    store_attr(self, 'stl_w,cnt_w,tv_w')
    self.metrics = L(LossMetrics(['stl', 'cnt', 'tv']))

  def __call__(self, pred, targ, fts, **kwargs):
    self.stl = self.stl_w*self.stl_loss(pred, targ, fts)
    self.cnt = self.cnt_w*self.cnt_loss(pred, targ, fts)
    self.tv  = self.tv_w *self.tv_loss (pred, targ, fts)
    return self.stl+self.cnt+self.tv
  
  @delegates(style_loss)
  def stl_loss(self,pred,targ,fts,**kwargs): return style_loss(pred,targ,fts,**kwargs)
  @delegates(content_loss)
  def cnt_loss(self,pred,targ,fts,**kwargs): return content_loss(pred,targ,fts,**kwargs)
  @delegates(tv_loss)
  def tv_loss(self,pred,targ,fts,**kwargs): return tv_loss(pred,targ,fts,**kwargs)

In [ ]:
#export
@funcs_kwargs
class SuperResLoss:
  _methods=['pixel_loss','cnt_loss']
  def __init__(self, **kwargs):
    self.metrics = L(LossMetrics(['pixel','cnt']))
    
  def __call__(self, pred, targ, fts,**kwargs):
    self.pixel = self.pixel_loss(pred,targ,fts,**kwargs)
    self.cnt   = self.cnt_loss  (pred,targ,fts,**kwargs)
    return self.pixel+self.cnt
    
  @delegates(pixel_loss)
  def pixel_loss(self,pred,targ,fts,**kwargs): return pixel_loss(pred,targ,fts,**kwargs)
  @delegates(content_loss)
  def cnt_loss(self,pred,targ,fts,**kwargs): return content_loss(pred,targ,fts,**kwargs)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_layer_features.ipynb.
Converted 03_loss.ipynb.
Converted 04_learner.ipynb.
Converted 04_models.ipynb.
Converted 06_callback.ipynb.
